In [7]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf

CNN_PATH = "ClimSim/baseline_models/CNN/model/"
MLP_PATH = "ClimSim/baseline_models/MLP/model/backup_phase-7_retrained_models_step2_lot-147_trial_0027.best.h5"

# cnn_model = tf.keras.layers.TFSMLayer(CNN_PATH, call_endpoint="serving_default")
mlp_model = tf.keras.models.load_model(MLP_PATH, compile=False)
mlp_model.summary()

Model: "retrained_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 124)]                0         []                            
                                                                                                  
 dense (Dense)               (None, 768)                  96000     ['input[0][0]']               
                                                                                                  
 leaky_re_lu (LeakyReLU)     (None, 768)                  0         ['dense[0][0]']               
                                                                                                  
 dense_1 (Dense)             (None, 640)                  492160    ['leaky_re_lu[0][0]']         
                                                                                    